### homework1：完成对facebook数据的k近邻的机器学习，并通过网格搜索找到最佳的kneighbors

In [1]:
import time
from sklearn.datasets import load_iris, fetch_20newsgroups, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

KNN

In [2]:
# K近邻
"""
K-近邻预测用户签到位置
:return:None
"""

# 读取数据
# 使用pandas的read_csv函数读取CSV文件，文件路径为"../data/FBlocation/train.csv"
data = pd.read_csv("../data/FBlocation/train.csv")

# 打印数据的前10行，以便了解数据的结构和内容
print(data.head(10))

# 打印数据的形状，即数据的行数和列数
print(data.shape)

# 打印数据的基本信息，包括每列的数据类型和非空值的数量
print(data.info())

# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
# 使用query方法筛选数据，只保留x在1.0到1.25之间且y在2.5到2.75之间的数据
data = data.query("x > 1.0 & x < 1.25 & y > 2.5 & y < 2.75")

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
(29118021, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29118021 entries, 0 to 29118020
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   row_id    int64  
 1   x         float64
 2   y         float64
 3   accuracy  int64  
 4   time      int64  
 5   place_id  int64  
dtypes: float64(2), int64(4)
memory usage: 1.3 GB
None


In [3]:
data.shape

(17710, 6)

In [4]:
data.describe()  # 用于生成数据集的描述性统计信息

,row_id,x,y,accuracy,time,place_id
count,1.771000e+04,17710.000000,17710.000000,17710.000000,17710.000000,1.771000e+04
mean,1.450569e+07,1.122538,2.632309,82.482101,397551.263128,5.129895e+09
std,8.353805e+06,0.077086,0.070144,113.613227,234601.097883,2.357399e+09
min,6.000000e+02,1.000100,2.500100,1.000000,119.000000,1.012024e+09
25%,7.327816e+06,1.049200,2.573800,25.000000,174069.750000,3.312464e+09
50%,1.443071e+07,1.123300,2.642300,62.000000,403387.500000,5.261906e+09
75%,2.163463e+07,1.190500,2.687800,75.000000,602111.750000,6.766325e+09
max,2.911215e+07,1.249900,2.749900,1004.000000,786218.000000,9.980711e+09


In [5]:
# 处理时间的数据，unit是秒，把秒转换成日期格式
# 使用pd.to_datetime函数将时间戳（以秒为单位）转换为日期时间格式
# data['time'] 是数据集中表示时间的列，unit='s' 表示时间戳的单位是秒
time_value = pd.to_datetime(data['time'], unit='s')

# 打印转换后的前10行时间数据，以便检查转换结果
print(time_value.head(10))  #最大时间是1月10号

600    1970-01-01 18:09:40
957    1970-01-10 02:11:10
4345   1970-01-05 15:08:02
4735   1970-01-06 23:03:03
5580   1970-01-09 11:26:50
6090   1970-01-02 16:25:07
6234   1970-01-04 15:52:57
6350   1970-01-01 10:13:36
7468   1970-01-09 15:26:06
8478   1970-01-08 23:52:02
Name: time, dtype: datetime64[ns]


In [6]:
# 把日期格式转换成 字典格式，把年，月，日，时，分，秒转换为字典格式，
# 将 time_value 转换为 Pandas 的 DatetimeIndex 对象
# DatetimeIndex 是 Pandas 中专门用于处理日期时间数据的索引类型
# 转换为 DatetimeIndex 后，可以更方便地进行时间相关的操作，如提取年、月、日等
time_value = pd.DatetimeIndex(time_value)

# 打印前10个时间值，检查转换结果
print(time_value[0:10])

--------------------------------------------------
DatetimeIndex(['1970-01-01 18:09:40', '1970-01-10 02:11:10',
               '1970-01-05 15:08:02', '1970-01-06 23:03:03',
               '1970-01-09 11:26:50', '1970-01-02 16:25:07',
               '1970-01-04 15:52:57', '1970-01-01 10:13:36',
               '1970-01-09 15:26:06', '1970-01-08 23:52:02'],
              dtype='datetime64[ns]', name='time', freq=None)


In [7]:
print('-' * 50)
# 构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
print(type(data))
# data['day'] = time_value.day
# data['hour'] = time_value.hour
# data['weekday'] = time_value.weekday
#日期，是否是周末，小时对于个人行为的影响是较大的(例如吃饭时间去饭店，看电影时间去电影院等),所以才做下面的处理
data.insert(data.shape[1], 'day', time_value.day)  #data.shape[1]是代表插入到最后的意思,一个月的哪一天
data.insert(data.shape[1], 'hour', time_value.hour)  #是否去一个地方打卡，早上，中午，晚上是有影响的
data.insert(data.shape[1], 'weekday', time_value.weekday)  #0代表周一，6代表周日，星期几

#
# 把时间戳特征删除
data = data.drop(['time'], axis=1)
print('-' * 50)
data.head()

--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
--------------------------------------------------


,row_id,x,y,accuracy,place_id,day,hour,weekday
600,600,1.2214,2.7023,17,6683426742,1,18,3
957,957,1.1832,2.6891,58,6683426742,10,2,5
4345,4345,1.1935,2.6550,11,6889790653,5,15,0
4735,4735,1.1452,2.6074,49,6822359752,6,23,1
5580,5580,1.0089,2.7287,19,1527921905,9,11,4


In [8]:
#星期天，实际weekday的值是6，星期一是0，星期二是1，星期三是2，星期四是3，星期五是4，星期六是5
per = pd.Period('2025-01-10 18:00', 'h')
per.weekday

4

In [9]:
#观察数据，看下是否有空值，异常值
data.describe()

,row_id,x,y,accuracy,place_id,day,hour,weekday
count,1.771000e+04,17710.000000,17710.000000,17710.000000,1.771000e+04,17710.000000,17710.000000,17710.000000
mean,1.450569e+07,1.122538,2.632309,82.482101,5.129895e+09,5.101863,11.485545,3.092377
std,8.353805e+06,0.077086,0.070144,113.613227,2.357399e+09,2.709287,6.932195,1.680218
min,6.000000e+02,1.000100,2.500100,1.000000,1.012024e+09,1.000000,0.000000,0.000000
25%,7.327816e+06,1.049200,2.573800,25.000000,3.312464e+09,3.000000,6.000000,2.000000
50%,1.443071e+07,1.123300,2.642300,62.000000,5.261906e+09,5.000000,12.000000,3.000000
75%,2.163463e+07,1.190500,2.687800,75.000000,6.766325e+09,7.000000,17.000000,4.000000
max,2.911215e+07,1.249900,2.749900,1004.000000,9.980711e+09,10.000000,23.000000,6.000000


In [10]:
# 把签到数量少于n个目标位置删除，place_id是标签，即目标值
place_count = data.groupby('place_id').count()
place_count

,row_id,x,y,accuracy,day,hour,weekday
place_id,,,,,,,
1012023972,1,1,1,1,1,1,1
1057182134,1,1,1,1,1,1,1
1059958036,3,3,3,3,3,3,3
1085266789,1,1,1,1,1,1,1
1097200869,1044,1044,1044,1044,1044,1044,1044
...,...,...,...,...,...,...,...
9904182060,1,1,1,1,1,1,1
9915093501,1,1,1,1,1,1,1
9946198589,1,1,1,1,1,1,1


In [11]:
# place_count['x'] 可以任意选取一列都一样，这里选取x列，还可以选取y列等
place_count['x'].describe()

count     805.000000
mean       22.000000
std        88.955632
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max      1044.000000
Name: x, dtype: float64

In [12]:
# # 把index变为0,1,2，3,4,5,6这种效果，从零开始排，原来的index是row_id
#只选择去的人大于3的数据，认为1,2,3的是噪音，这个地方去的人很少，不用推荐给其他人
tf = place_count[place_count.row_id > 3].reset_index()
tf  #剩余的签到地点

,place_id,row_id,x,y,accuracy,day,hour,weekday
0,1097200869,1044,1044,1044,1044,1044,1044,1044
1,1228935308,120,120,120,120,120,120,120
2,1267801529,58,58,58,58,58,58,58
3,1278040507,15,15,15,15,15,15,15
4,1285051622,21,21,21,21,21,21,21
...,...,...,...,...,...,...,...,...
234,9741307878,5,5,5,5,5,5,5
235,9753855529,21,21,21,21,21,21,21
236,9806043737,6,6,6,6,6,6,6
237,9809476069,23,23,23,23,23,23,23


In [16]:
# 根据设定的地点目标值，对原本的样本进行过滤
# isin可以过滤某一列要在一组值

# 对数据集 data 进行筛选，只保留 'place_id' 列的值在 tf.place_id 中的行
# isin() 方法用于判断某一列的值是否在指定的列表中
# tf.place_id 是一个包含目标 place_id 的列表或 Series
data = data[data['place_id'].isin(tf.place_id)]
data.shape

(16918, 8)

In [76]:
type(data)

pandas.core.frame.DataFrame

In [81]:
# # 取出数据当中的特征值和目标值
y = data['place_id']
# 删除目标值，保留特征值，
x = data.drop(['place_id'], axis=1)
# 删除无用的特征值，row_id是索引,这就是噪音
x = x.drop(['row_id'], axis=1)
# 打印特征值的形状，查看剩余的特征数量
print(x.shape)
# 打印目标值的形状，查看目标值数量
print(y.shape)
# 打印特征值的列名，查看剩余的特征
print(x.columns)

(16918, 6)
(16918,)
Index(['x', 'y', 'accuracy', 'day', 'hour', 'weekday'], dtype='object')


In [23]:
data

,row_id,x,y,accuracy,place_id,day,hour,weekday
600,600,1.2214,2.7023,17,6683426742,1,18,3
957,957,1.1832,2.6891,58,6683426742,10,2,5
4345,4345,1.1935,2.6550,11,6889790653,5,15,0
4735,4735,1.1452,2.6074,49,6822359752,6,23,1
5580,5580,1.0089,2.7287,19,1527921905,9,11,4
...,...,...,...,...,...,...,...,...
29100203,29100203,1.0129,2.6775,12,3312463746,1,10,3
29108443,29108443,1.1474,2.6840,36,3533177779,7,23,2
29109993,29109993,1.0240,2.7238,62,6424972551,8,15,3
29111539,29111539,1.2032,2.6796,87,3533177779,4,0,6


In [82]:
x

,x,y,accuracy,day,hour,weekday
600,1.2214,2.7023,17,1,18,3
957,1.1832,2.6891,58,10,2,5
4345,1.1935,2.6550,11,5,15,0
4735,1.1452,2.6074,49,6,23,1
5580,1.0089,2.7287,19,9,11,4
...,...,...,...,...,...,...
29100203,1.0129,2.6775,12,1,10,3
29108443,1.1474,2.6840,36,7,23,2
29109993,1.0240,2.7238,62,8,15,3
29111539,1.2032,2.6796,87,4,0,6


In [25]:
y

600         6683426742
957         6683426742
4345        6889790653
4735        6822359752
5580        1527921905
               ...    
29100203    3312463746
29108443    3533177779
29109993    6424972551
29111539    3533177779
29112154    4932578245
Name: place_id, Length: 16918, dtype: int64

In [18]:
# 进行数据的分割训练集合测试集 特征矩阵（X）、目标值（y）
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
print(std.mean_)
print(std.var_)
x_test = std.transform(x_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化
print('-' * 50)
# print(id(x_test))
print(std.mean_)
print(std.var_)

[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]
--------------------------------------------------
[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]


In [83]:
x.shape

(16918, 6)

In [26]:
x_train.shape

(12688, 6)

In [30]:
# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=6)

# # fit， predict,score，训练，knn的fit是不训练的，只是把训练集的特征值和目标值放入到内存中
knn.fit(x_train, y_train)
# # #
# # # 得出预测结果
y_predict = knn.predict(x_test)
# #
print("预测的目标签到位置为：", y_predict[0:10])
# # #
# # # # 得出准确率,是评估指标
print("预测的准确率:", knn.score(x_test, y_test))
# print(y_predict)
# y_test

预测的目标签到位置为： [5689129232 1097200869 2355236719 9632980559 6424972551 4022692381
 8048985799 6683426742 1435128522 3312463746]
预测的准确率: 0.484160756501182


In [31]:
print(y_test[0:10])

16751286    1893548673
12423167    1097200869
7517023     6097504486
4400015     9632980559
26212472    6424972551
7089828     4022692381
10935607    2327054745
25025511    3533177779
27755137    1435128522
19678934    3312463746
Name: place_id, dtype: int64


网格搜索

In [33]:
# 网格搜索时讲解
# 构造一些参数（超参）的值进行搜索
# n_neighbors：KNN 中的 K 值，尝试 [3, 5, 10, 12, 15]
# weights：权重计算方式，尝试 'uniform'（均匀权重）和 'distance'（距离权重）
param = {"n_neighbors": [3, 5, 10, 12, 15], 'weights': ['uniform', 'distance']}

# 进行网格搜索
# knn：KNN 模型对象
# param_grid：超参数网格，指定需要搜索的参数组合
# cv=3：3 折交叉验证，用其中 2 折训练，1 折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

# 使用训练集进行网格搜索
# gc.fit 会将 x_train 进一步分为训练集和验证集，进行交叉验证
gc.fit(x_train, y_train)

# 预测准确率，为了给大家看看
# 使用测试集评估最佳模型的性能
print("在测试集上准确率：", gc.score(x_test, y_test))

# 打印交叉验证中最好的结果
# best_score_：交叉验证中最好的平均得分
print("在交叉验证当中最好的结果：", gc.best_score_)

# 打印选择的最好的模型
# best_estimator_：最佳模型对象，包含最优的超参数
print("选择最好的模型是：", gc.best_estimator_)

# 打印每个超参数组合在交叉验证中的结果
# cv_results_：包含每次交叉验证的详细结果
print("每个超参数每次交叉验证的结果：")
print(gc.cv_results_)

E:\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


在测试集上准确率： 0.49763593380614657
在交叉验证当中最好的结果： 0.4816362349278435
选择最好的模型是： KNeighborsClassifier(n_neighbors=12, weights='distance')
每个超参数每次交叉验证的结果：
{'mean_fit_time': array([0.00923196, 0.00897034, 0.00896947, 0.01063108, 0.00896994,
       0.00897002, 0.00896939, 0.00897034, 0.00897662, 0.00930214]), 'std_fit_time': array([1.14973706e-03, 2.24783192e-07, 8.14004054e-04, 1.69384554e-03,
       1.12391596e-07, 3.89335909e-07, 2.24783192e-07, 2.97360213e-07,
       9.83474632e-06, 4.69515903e-04]), 'mean_score_time': array([0.07364472, 0.03488294, 0.0789837 , 0.05332422, 0.08615367,
       0.0554812 , 0.09368714, 0.06278992, 0.09666991, 0.06843797]), 'std_score_time': array([0.00163524, 0.00162772, 0.00089096, 0.00672431, 0.00102679,
       0.00046952, 0.00215295, 0.00162742, 0.00081426, 0.00124316]), 'param_n_neighbors': masked_array(data=[3, 3, 5, 5, 10, 10, 12, 12, 15, 15],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
      

### homework2：完成对20类新闻数据集，通过贝叶斯分析，实现分类预测，理解贝叶斯公式计算方法，得出，精确率，召回率，F1-score,并理解计算某个分类的AUC值的操作

In [34]:
"""
朴素贝叶斯进行文本分类
:return: None
"""
news = fetch_20newsgroups(subset='all', data_home='../data')

print(len(news.data))  #样本数，包含的特征
print('-' * 50)
print(news.data[0])  #第一个样本 特征
print('-' * 50)
print(news.target[0:5])  #标签
print(np.unique(news.target))  #标签的类别
print(news.target_names)  #标签的名字

18846
--------------------------------------------------
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


----------------------------------------

In [84]:
print('-' * 50)
# 进行数据分割
x_train, x_test, y_train, y_test = train_test_split(news.data, news.target, test_size=0.25, random_state=1)

# 对数据集进行特征抽取
tf = TfidfVectorizer()

# 以训练集当中的词的列表进行每篇文章重要性统计['a','b','c','d']
x_train = tf.fit_transform(x_train)
#针对特征内容，可以自行打印，下面的打印可以得到特征数目，总计有15万特征
print(len(tf.get_feature_names_out()))

--------------------------------------------------
153196


In [38]:
print(tf.get_feature_names_out()[100000])

murky


In [39]:
print(tf.get_feature_names_out()[0:10])

['00' '000' '0000' '00000' '0000000004' '0000000005' '0000000667'
 '0000001200' '000003' '000005102000']


In [40]:
print(tf.get_feature_names_out()[100000:100000 + 10])

['murky' 'murmurs' 'murnane' 'murph' 'murphey' 'murphy' 'murr11' 'murray'
 'murray_craven' 'murrayfield']


In [90]:
# 进行朴素贝叶斯算法的预测,alpha是拉普拉斯平滑系数，分子和分母加上一个系数，分母加alpha*特征词数目
mlt = MultinomialNB(alpha=1.0)

# print(x_train.toarray())
# 训练
start = time.time()
mlt.fit(x_train, y_train)  #训练模型
end = time.time()
end - start  #统计训练时间

0.0627903938293457

In [42]:
x_transform_test = tf.transform(x_test)  #特征数目不发生改变
print(len(tf.get_feature_names_out()))  #查看特征数目

153196


In [43]:
x_transform_test.shape

(4712, 153196)

In [44]:
start = time.time()
y_predict = mlt.predict(x_transform_test)

print("预测的前面10篇文章类别为：", y_predict[0:10])

# 得出准确率,这个是很难提高准确率，为什么呢？
print("准确率为：", mlt.score(x_transform_test, y_test))
end = time.time()
end - start  #预测时间

预测的前面10篇文章类别为： [16 19 18  1  9 15  1  2 16 13]
准确率为： 0.8518675721561969


0.03687691688537598

In [45]:
#预测的文章数目
len(y_predict)

4712

In [46]:
# 目前这个场景我们不需要召回率，support是真实的为那个类别的有多少个样本 即每个类别的样本数量
print(classification_report(y_test, y_predict,
                            target_names=news.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.91      0.77      0.83       199
           comp.graphics       0.83      0.79      0.81       242
 comp.os.ms-windows.misc       0.89      0.83      0.86       263
comp.sys.ibm.pc.hardware       0.80      0.83      0.81       262
   comp.sys.mac.hardware       0.90      0.88      0.89       234
          comp.windows.x       0.92      0.85      0.88       230
            misc.forsale       0.96      0.67      0.79       257
               rec.autos       0.90      0.87      0.88       265
         rec.motorcycles       0.90      0.95      0.92       251
      rec.sport.baseball       0.89      0.96      0.93       226
        rec.sport.hockey       0.95      0.98      0.96       262
               sci.crypt       0.76      0.97      0.85       257
         sci.electronics       0.84      0.80      0.82       229
                 sci.med       0.97      0.86      0.91       249
         

In [47]:
y_test.shape  #测试集中有多少 样本

(4712,)

In [48]:
# 将 y_test 中等于 0 的标签替换为 1，其余替换为 0
# np.where(condition, x, y)：满足 condition 的位置返回 x，否则返回 y
y_test1 = np.where(y_test == 0, 1, 0)
print(y_test1.sum())  #label为0的样本数

199


In [49]:
y_predict1 = np.where(y_predict == 0, 1, 0)
print(y_predict1.sum())

168


In [50]:
print(y_test1[0:20])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]


In [51]:
y_predict1[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [52]:
#168个预测为正例中，有153预测正确了
(y_test1 * y_predict1).sum()

np.int64(153)

In [53]:
# 把0-19总计20个分类，变为0和1
# 5是可以改为0到19的
y_test1 = np.where(y_test == 5, 1, 0)
print(y_test1.sum())  #label为5的样本数
y_predict1 = np.where(y_predict == 5, 1, 0)
print(y_predict1.sum())
# roc_auc_score的y_test只能是二分类,针对多分类如何计算AUC
print("AUC指标：", roc_auc_score(y_test1, y_predict1))

230
214
AUC指标： 0.924078924393225


In [54]:
#算多分类的精确率，召回率，F1-score
FP = np.where((np.array(y_test1) - np.array(y_predict1)) == -1, 1, 0).sum()  #FP是18
TP = y_predict1.sum() - FP  #TP是196
print(TP)
FN = np.where((np.array(y_test1) - np.array(y_predict1)) == 1, 1, 0).sum()  #FN是34
print(FN)  #FN是1
TN = np.where(y_test1 == 0, 1, 0).sum() - FP  #4464
print(TN)

196
34
4464


![本地图片示例](./images/confusion_matrix.png)

In [55]:
TP / (TP + FP)  #精确率

np.float64(0.9158878504672897)

In [56]:
TP / (TP + FN)  #召回率

np.float64(0.8521739130434782)

In [57]:
#F1-score
2 * TP / (2 * TP + FP + FN)

np.float64(0.8828828828828829)

### homework3：完成决策树的练习

In [92]:
"""
决策树对泰坦尼克号进行预测生死
:return: None
"""
# 获取数据
titan = pd.read_csv("../data/titanic.txt")
titan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row.names  1313 non-null   int64  
 1   pclass     1313 non-null   object 
 2   survived   1313 non-null   int64  
 3   name       1313 non-null   object 
 4   age        633 non-null    float64
 5   embarked   821 non-null    object 
 6   home.dest  754 non-null    object 
 7   room       77 non-null     object 
 8   ticket     69 non-null     object 
 9   boat       347 non-null    object 
 10  sex        1313 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 113.0+ KB


In [93]:
titan

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male
...,...,...,...,...,...,...,...,...,...,...,...
1308,1309,3rd,0,"Zakarian, Mr Artun",NaN,NaN,NaN,NaN,NaN,NaN,male
1309,1310,3rd,0,"Zakarian, Mr Maprieder",NaN,NaN,NaN,NaN,NaN,NaN,male
1310,1311,3rd,0,"Zenn, Mr Philip",NaN,NaN,NaN,NaN,NaN,NaN,male
1311,1312,3rd,0,"Zievens, Rene",NaN,NaN,NaN,NaN,NaN,NaN,female


In [94]:
# 处理数据，找出特征值和目标值
# 特征值
x = titan[['pclass', 'age', 'sex']]
# 目标值
y = titan['survived']
print(x.info())  # 用来判断是否有空值
x.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     633 non-null    float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB
None


,pclass,age,sex
count,1313,633.000000,1313
unique,3,NaN,2
top,3rd,NaN,male
freq,711,NaN,850
mean,NaN,31.194181,NaN
std,NaN,14.747525,NaN
min,NaN,0.166700,NaN
25%,NaN,21.000000,NaN
50%,NaN,30.000000,NaN
75%,NaN,41.000000,NaN


In [95]:
x.loc[:, 'age'].max()

np.float64(71.0)

In [96]:
# 一定要进行缺失值处理,填为均值
mean = x['age'].mean()
print(mean)
x.loc[:, 'age'] = x.loc[:, 'age'].fillna(mean)

31.19418104265403


In [97]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  1313 non-null   object 
 1   age     1313 non-null   float64
 2   sex     1313 non-null   object 
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [98]:
# 分割数据集到训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=4)
print(x_train.head())

    pclass        age     sex
598    2nd  30.000000    male
246    1st  62.000000    male
905    3rd  31.194181  female
300    1st  31.194181  female
509    2nd  64.000000    male


In [99]:
type(x_train)

pandas.core.frame.DataFrame

In [100]:
sum(y_train)

334

In [101]:
#性别是女性的数量
x_train[x_train['sex'] == 'female'].count()

pclass    341
age       341
sex       341
dtype: int64

In [102]:
y_train

598     0
246     0
905     0
300     0
509     0
       ..
360     0
709     0
439     0
174     0
1146    0
Name: survived, Length: 984, dtype: int64

In [103]:
#女性中存活的情况对比
z = x_train.copy()  #z是为了把特征和目标存储到一起
z['survived'] = y_train  #把目标值存储到z中
z[z['sex'] == 'female']['survived'].value_counts()  #女性中存活的情况

survived
1    230
0    111
Name: count, dtype: int64

In [104]:
z[z['sex'] == 'male']['survived'].value_counts()

survived
0    539
1    104
Name: count, dtype: int64

In [105]:
y_train.value_counts()  #没存活的是650，存活的是334

survived
0    650
1    334
Name: count, dtype: int64

In [106]:
x_train.loc[:, 'sex'].value_counts()

sex
male      643
female    341
Name: count, dtype: int64

In [107]:
#查看未存活的人的数量
x_train

,pclass,age,sex
598,2nd,30.000000,male
246,1st,62.000000,male
905,3rd,31.194181,female
300,1st,31.194181,female
509,2nd,64.000000,male
...,...,...,...
360,2nd,31.194181,male
709,3rd,28.000000,male
439,2nd,34.000000,male
174,1st,46.000000,male


In [108]:
x_train.to_dict(orient="records")  #把df变为字典，样本变为一个一个的字典，字典中列名变为键

[{'pclass': '2nd', 'age': 30.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 62.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '2nd', 'age': 64.0, 'sex': 'male'},
 {'pclass': '1st', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 24.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 21.0, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '2nd', 'age': 23.0, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'male'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '3rd', 'age': 31.19418104265403, 'sex': 'female'},
 {'pclass': '1st', 'age': 4

In [109]:
# 进行处理（特征工程）特征-》类别-》one_hot编码
dict = DictVectorizer(sparse=False)

# 这一步是对字典进行特征抽取,to_dict可以把df变为字典，records代表列名变为键
x_train = dict.fit_transform(x_train.to_dict(orient="records"))
print(type(x_train))
print(dict.get_feature_names_out())
print('-' * 50)
x_test = dict.transform(x_test.to_dict(orient="records"))
print(x_train)

<class 'numpy.ndarray'>
['age' 'pclass=1st' 'pclass=2nd' 'pclass=3rd' 'sex=female' 'sex=male']
--------------------------------------------------
[[30.          0.          1.          0.          0.          1.        ]
 [62.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          1.          0.        ]
 ...
 [34.          0.          1.          0.          0.          1.        ]
 [46.          1.          0.          0.          0.          1.        ]
 [31.19418104  0.          0.          1.          0.          1.        ]]


In [110]:
# 用决策树进行预测，修改max_depth试试,修改criterion为entropy
# 树过于复杂，就会产生过拟合
dec = DecisionTreeClassifier()

#训练
dec.fit(x_train, y_train)

# 预测准确率
print("预测的准确率：", dec.score(x_test, y_test))

# 导出决策树的结构
export_graphviz(dec, out_file="tree.dot",
                feature_names=['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'female', 'male'])

预测的准确率： 0.8085106382978723
